In [1]:
import os
import csv
import re

In [2]:

root_path

'd:\\Projects\\Py-Projects\\folder-master'

In [3]:
root_path = os.getcwd()

language_folders = [folder for folder in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, folder))]
language_folders

['.git', 'target-folder', 'material-folder']

In [4]:
target_folder = "target-folder"  # 目标文件夹路径

# 获取所有语言分类子文件夹的路径
language_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]
language_folders

['English_EN', 'French_FR', 'German_DE', 'Dutch_NL']